In [149]:
import numpy as np
import torch as t
from torch.distributions import Normal, Categorical, Bernoulli
from torch.distributions import MultivariateNormal as MvNormal
import matplotlib.pyplot as plt
%matplotlib widget
from ipywidgets import FloatSlider, IntSlider, interact, interact_manual

$$
\newcommand{\bracket}[3]{\left#1 #3 \right#2}
\newcommand{\b}{\bracket{(}{)}}
\newcommand{\Bernoulli}{{\rm Bernoulli}\b}
\newcommand{\Categorical}{{\rm Categorical}\b}
\newcommand{\x}{\mathbf{x}}
\newcommand{\X}{\mathbf{X}}
\newcommand{\m}{\boldsymbol{\mu}}
\newcommand{\P}{{\rm P}\b}
\newcommand{\dd}[2][]{\frac{\partial #1}{\partial #2}}
\newcommand{\S}{\mathbf{\Sigma}}
\newcommand{\Sh}{\mathbf{\hat{\Sigma}}}
\newcommand{\mh}{\boldsymbol{\hat{\mu}}}
\newcommand{\N}{\mathcal{N}\b}
\newcommand{\det}{\bracket{\lvert}{\rvert}}
\newcommand{\sb}{\bracket{[}{]}}
\newcommand{\E}{\mathbb{E}\sb}
\newcommand{\Var}{{\rm Var}\sb}
\newcommand{\Cov}{{\rm Cov}\sb}
\DeclareMathOperator*{\argmax}{arg\,max}
\DeclareMathOperator*{\argmin}{arg\,min}
\newcommand{\ph}{\hat{p}}
\newcommand{\at}{\bracket{.}{\rvert}}
\newcommand{\w}{\mathbf{w}}
\newcommand{\W}{\mathbf{W}}
\newcommand{\W}{\mathbf{W}}
\newcommand{\Wh}{\mathbf{\hat{W}}}
\newcommand{\Y}{\mathbf{Y}}
\newcommand{\L}{\mathcal{L}}
\newcommand{\wh}{\mathbf{\hat{w}}}
\newcommand{\y}{\mathbf{y}}
\newcommand{\0}{\mathbf{0}}
\newcommand{\I}{\mathbf{I}}
\newcommand{\La}{\mathbf{\Lambda}}
\newcommand{\S}{\mathbf{\Sigma}}
\newcommand{\Sprior}{\S_\text{prior}}
\newcommand{\Spost}{\S_\text{post}}
\newcommand{\mprior}{\m_\text{prior}}
\newcommand{\mpost}{\m_\text{post}}
\newcommand{\Xt}{\tilde{\X}}
\newcommand{\yt}{\tilde{\y}}
\newcommand{\p}{\mathbf{p}}
\newcommand{\l}{\boldsymbol{\ell}}
\DeclareMathOperator{\softmax}{softmax}
\newcommand{\z}{\mathbf{z}}
\newcommand{\norm}{\bracket{\lVert}{\rVert}}
$$

<h1> Lecture 5: Unsupervised learning and clustering </h1>

<h2> Clustering/unsupervised learning is not classification/supervised learning (1/2) </h2>
Clustering is a type of unsupervised learning, which is very, very different from the supervised learning you have seen in my slides so far.

In supervised learning, we have a list of input, `x`, output, `y`, pairs as data, and the idea is to learn a function that maps from a new input test point, to a distribution over the corresonding `Y`

```
#### Supervised learning
x : X # Input
y : Y # Output
    [(X, Y)] -> (X -> Y) 
    [(X, Y)] -> (X -> Distribution{Y}) 
```

In unsupervised learning/clustering, we only have a list of inputs, and the goal is to compute a (distribution over) a latent variable that somehow summarises the structure of the inputs.

```
#### Unsupervised learning
x : X # Input
z : Z # Latent
    [X] -> [Z]
    [X] -> [Distribution{Z}]
```

<h2> First example </h2>

In [150]:
X = t.cat([
    t.randn(50, 2)/5 + t.tensor([1., 0.]),
    t.randn(50, 2)/5 + t.tensor([-1., 0.])
])

Z = t.cat([
    t.zeros(50).int(),
    t.ones(50).int()
])

#fig = plot.figure()

fig, axs = plt.subplots(ncols=3, figsize=(9,3), sharey=True)
axs[0].set_xlim(-2, 2)
axs[1].set_xlim(-2, 2)
axs[2].set_xlim(-2, 2)
axs[0].set_ylim(-2, 2)
axs[0].scatter(X[:, 0], X[:, 1])
axs[1].scatter(X[:, 0], X[:, 1], c=Z);
axs[2].scatter(X[:, 0], X[:, 1], c=-Z);
axs[0].set_title("original data, without labels")
axs[1].set_title("clustered data")
axs[2].set_title("equiv. clustering");

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<h2> Clustering/unsupervised learning is not classification/supervised learning (2/2) </h2>



In [151]:
Y = Bernoulli(t.sigmoid(100*X[:, 1])).sample()

fig, ax = plt.subplots()
ax.set_xlim(-2, 2)
ax.set_ylim(-1, 1)
ax.set_xlabel("height")
ax.set_ylabel("A-level scores")
ax.scatter(X[:, 0], X[:, 1], c=Y);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<h2> Simplest clustering algorithm: K-means </h2>

There are parameters, $\m$, and latent variables, $z_\lambda$, associated with each datapoint.

The parameters, $\m$ describe the central points of $K$ clusters, and the latents assign each datapoint to a cluster.  Thus, we have three/four key objects,
```
X.shape  = (N, 1, D)
mu.shape =    (K, D)
p.shape  = (N, K, 1)
z.shape  = (N,)
```
where `N` is the number of datapoints, `D` is the dimension of each datapoint, and `p` is the one-hot representation of `z`.

The objective is to minimise the squared distance between each datapoint and its assigned cluster center,

\begin{align}
  \L(\z, \m) &= \sum_\lambda \norm{\x_\lambda - \m_{z_\lambda}}^2
\end{align}

To optimize this algorithm, we use coordinate descent (i.e. we alternate between optimizing $\z$ and $\m$).

First, we assign each datapoint to the nearest cluster,

\begin{align}
  z_\lambda &\rightarrow \argmin_k \norm{\x_\lambda - \m_k}^2
\end{align}

Then, we put the cluster centers at the mean of the datapoints,

\begin{align}
  \m_k &\rightarrow \frac{1}{\sum_\lambda \delta_{k, z_\lambda}} \sum_{\lambda} \delta_{k, z_\lambda} \x_\lambda
\end{align}

In [191]:
t.manual_seed(0)

X = t.cat([
    t.randn(50, 2)/5 + t.tensor([1., 0.]),
    t.randn(50, 2)/5,
    t.randn(50, 2)/5 + t.tensor([-1., 0.])
])

def kmeans(X, K):
    N, D = X.shape
    X = X[:, None, :]
    mu = t.randn(K, D)
    while True:
        sd = ((X - mu)**2).sum(-1) # sd.shape = (N, K)
        z = t.argmin(sd, 1)        # z.shape  = (N)
        q = t.zeros(N, K, 1)
        q[t.arange(N), z, 0] = 1.
        print(f"loss = {loss(X, z, mu).item()}")
        plot(X, z, mu)
        yield None
        
        mu = (q * X).sum(0) / q.sum(0)
        print(f"loss = {loss(X, z, mu).item()}")
        plot(X, z, mu)
        yield None
        
def loss(X, z, mu):
    return ((X - mu[z, :][:, None, :])**2).mean()
    
def plot(X, z, mu):
    fig, ax = plt.subplots()
    ax.set_xlim(-2, 2)
    ax.set_ylim(-2, 2)
    ax.scatter(X[:, 0, 0], X[:, 0, 1], c=z);
    ax.scatter(mu[:, 0], mu[:, 1], s=100, c='r', label="cluster centers")
    ax.legend()
    
kmeans_iter = iter(kmeans(X, 3))
def kmeans_call():
    next(kmeans_iter)
interact_manual(kmeans_call);

#Change marker type

interactive(children=(Button(description='Run Interact', style=ButtonStyle()), Output()), _dom_classes=('widge…

However, one strange property of this algorithm is that it assumes "hard" cluster assignments.

<h2> Soft clustering and the Gaussian Mixture Model (GMM) </h2>

The goal of the GMM, is really just to model the density of the data we use the latent variables to give us a better model of the data.

For instance, consider fitting a MvNormal to the following data,

In [148]:
X = t.cat([
    t.randn(50, 2)/5 + t.tensor([1., 0.]),
    t.randn(50, 2)/5 + t.tensor([-1., 0.])
])

N = X.shape[0]
mu = X.sum(0)/N
cov = ((X - mu).T @ (X-mu))/N
mvn = MvNormal(mu, cov)

logPx = mvn.log_prob(X).sum()
print(f"logPx = {logPx}")

P = 100
x0s = t.linspace(-2, 2, P)[:, None].expand(P, P)
x1s = x0s.T
xs  = t.stack([x0s.reshape(-1), x1s.reshape(-1)], 1)
ps = mvn.log_prob(xs).exp().view(P, P)

fig, ax = plt.subplots()
ax.set_xlim(-2, 2)
ax.set_ylim(-2, 2)
ax.scatter(X[:, 0], X[:, 1]);
ax.contour(x0s, x1s, ps);

logPx = -120.26077270507812


/Users/laurence/programs/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:20: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

How do we get a better model for this kind of data?  We use a GMM.

The simplest form of a GMM is a density, with $K$ different Gaussians, in different locations, $\m_k$, and with different covariance matrices, $\S_k$,

\begin{align}
  \P{x_\lambda} &= \sum_k p_k \N{\x_\lambda; \m_k, \S_k}.
\end{align}

However, this doesn't give us a way to extract any notion of assignment of a datapoint to a mixture component.

We therefore write this distribution in terms of a latent variable,

\begin{align}
  \P{\x_\lambda} &= \sum_{z_\lambda} \P{\x_\lambda| z_\lambda} \P{z_\lambda}
\end{align}

where, $\P{z_\lambda}$ is a Categorical, and represents the mixture component for the $\lambda$th data point, and $\P{x_\lambda| z_\lambda}$ is a _single_ Gaussian, (corresponding to the $z_\lambda$th mixture component),

\begin{align}
  \P{z_\lambda} &= \Categorical{z_\lambda; \p}\\
  \P{\x_\lambda| z_\lambda} &= \N{\x_\lambda; \m_{z_\lambda}, \S_{z_\lambda}}.
\end{align}

The substituting in the values of $\P{z_\lambda}$ and $\P{x_\lambda| z_\lambda}$, we get the same expression as above, except use $z_\lambda$ as the summation index, instead of $k$

\begin{align}
  \P{\x_\lambda} &= \sum_{z_\lambda} p_{z_\lambda} \N{\x_\lambda; \m_{z_\lambda}, \S_{z_\lambda}}.
\end{align}



Now, we can use Bayes theorem to give a posterior distribution over $z_\lambda$,

\begin{align}
  \P{z_\lambda| \x_\lambda} &\propto \P{\x_\lambda| z_\lambda} \P{z_\lambda}
\end{align}

Suggests an algorithm similar to K-means: first compute the posterior over $z_\lambda$, then update the parameters of the Gaussians, $\m_k$ and $\S_k$, using a mean weighted by the posterior.

This algorith is known as expectation-maximization (EM).  Computing the posterior is known as the E-step, and updating the parameters is the M-step.

In [189]:
t.manual_seed(0)

X = t.cat([
    t.randn(50, 2)/3 + t.tensor([1., 0.]),
    t.randn(50, 2)/3 + t.tensor([-1., 0.])
])

def gmm_em(X, K):
    N, D = X.shape
    X = X[:, None, :]
    mu  = t.randn(K, D)
    cov = 0.5*t.eye(D, D).expand(K, -1, -1)
    while True:
        # unnormalized posterior probability
        q = MvNormal(mu, cov).log_prob(X).exp()
        # normalized posterior probability
        q = q / q.sum(1, keepdim=True)
        # expand out last such that q.shape = (N, K, 1)
        q = q[:, :,  None]
        plot_gmm(X, q, mu, cov)
        yield None
        
        #weighted mean
        mu = (q * X).sum(0) / q.sum(0)
        plot_gmm(X, q, mu, cov)
        yield None

        
def plot_gmm(X, q, mu, cov):
    fig, ax = plt.subplots()
    ax.set_xlim(-2, 2)
    ax.set_ylim(-2, 2)
    ax.scatter(X[:, 0, 0], X[:, 0, 1], c=q[:, 0, 0]);
    ax.scatter(mu[:, 0], mu[:, 1], s=100, c='r', label="cluster centers")
    ax.legend()

    
gmm_em_iter = iter(gmm_em(X, 2))
def gmm_em_call():
    next(gmm_em_iter)
interact_manual(gmm_em_call);

#Change marker type

interactive(children=(Button(description='Run Interact', style=ButtonStyle()), Output()), _dom_classes=('widge…

But what is the objective function in EM?

Turns out that this is a very subtle question.

\begin{align}
  \L{\m, \q} &= \
\end{align}